In [1]:
import requests
import time

INPUT_FILE = "/Users/saanviaima/Documents/GitHub/Hyrbid-Alignment/HMM-iter1-scores-table.txt"
OUTPUT_FASTA = "all_sequences.fasta"
NOT_FOUND_LOG = "not_found.txt"
NAME_MAP_FILE = "name_map.txt"
ORGANISM_ID = 9606
PAUSE_BETWEEN_REQUESTS = 0.3

def extract_gene_names(file_path):
    def extract_middle_from_name(name):
        parts = name.split('_')
        if len(parts) >= 3:
            return parts[1]
        elif len(parts) == 2:
            return parts[1]
        return None

    gene_names = []
    name_map = {}
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():
                original_name = line.split()[0]
                gene = extract_middle_from_name(original_name)
                if gene:
                    gene_names.append(gene)
                    name_map[original_name] = gene
    return list(set(gene_names)), name_map

def get_uniprot_accession(gene_name):
    url = (
        f"https://rest.uniprot.org/uniprotkb/search?"
        f"query=gene:{gene_name}+AND+organism_id:{ORGANISM_ID}+AND+reviewed:true"
        f"&fields=accession&format=tsv"
    )
    r = requests.get(url)
    if r.status_code == 200:
        lines = r.text.strip().split('\n')
        if len(lines) > 1:
            return lines[1].split('\t')[0]
    return None

def download_fasta(accession):
    url = f"https://rest.uniprot.org/uniprotkb/{accession}.fasta"
    r = requests.get(url)
    if r.status_code == 200:
        return r.text
    return None

gene_names, name_map = extract_gene_names(INPUT_FILE)
print(f"Found {len(gene_names)} unique gene names.")

with open(NAME_MAP_FILE, 'w') as map_file:
    for original, gene in name_map.items():
        map_file.write(f"{original} -> {gene}\n")

with open(OUTPUT_FASTA, 'w') as fasta_out, open(NOT_FOUND_LOG, 'w') as log_out:
    for gene in gene_names:
        print(f"Fetching {gene}...", end='')
        accession = get_uniprot_accession(gene)
        if accession:
            sequence = download_fasta(accession)
            if sequence:
                fasta_out.write(sequence)
                print(f" ✓ ({accession})")
            else:
                print(" ✗ (FASTA not found)")
                log_out.write(f"{gene}\t{accession}\tFASTA not found\n")
        else:
            print(" ✗ (No accession)")
            log_out.write(f"{gene}\tNOT FOUND\n")
        time.sleep(PAUSE_BETWEEN_REQUESTS)


Found 484 unique gene names.
Fetching FGFR4... ✓ (P22455)
Fetching PSKH2... ✓ (Q96QS6)
Fetching MAP3K5... ✓ (Q99683)
Fetching EIF2AK1... ✓ (Q9BQI3)
Fetching PRKCH... ✓ (P24723)
Fetching DAPK3... ✓ (O43293)
Fetching GRK5... ✓ (P34947)
Fetching CSNK2A1... ✓ (P68400)
Fetching SIK1B... ✗ (No accession)
Fetching NEK9... ✓ (Q8TD19)
Fetching MAPK9... ✓ (P45984)
Fetching CDK11A... ✓ (Q9UQ88)
Fetching NEK3... ✓ (P51956)
Fetching BMP2K... ✓ (Q9NSY1)
Fetching CAMK2A... ✓ (Q9UQM7)
Fetching GAK... ✓ (O14976)
Fetching GUCY2D... ✓ (Q02846)
Fetching OXSR1... ✓ (O95747)
Fetching SBK3... ✓ (P0C264)
Fetching BTK... ✓ (Q06187)
Fetching TXK... ✓ (P42681)
Fetching PRKD2... ✓ (Q9BZL6)
Fetching ABL2... ✓ (P42684)
Fetching PRKAA2... ✓ (P54646)
Fetching MAPK11... ✓ (Q15759)
Fetching TLK2... ✓ (Q86UE8)
Fetching ACVR2A... ✓ (P27037)
Fetching LMTK2... ✓ (Q8IWU2)
Fetching TSSK4... ✓ (Q6SA08)
Fetching HUNK... ✓ (P57058)
Fetching CDK7... ✓ (P50613)
Fetching CAMKV... ✓ (Q8NCB2)
Fetching VRK2... ✓ (Q86Y07)
Fetching MYL

In [2]:
#GROUND TRUTH FORMATTER

import pandas as pd

input_file = "/Users/saanviaima/Documents/GitHub/Hyrbid-Alignment/Benchmark-aligned-residue-pairs-SE.out"
output_file = "clean_ground_truth.csv"

rows = []

with open(input_file, "r") as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) == 8:
            seq1, pos1, res1, _, seq2, pos2, res2, _ = parts
            rows.append([seq1, int(pos1), res1, seq2, int(pos2), res2, 1])

df = pd.DataFrame(rows, columns=["Seq1", "Pos1", "Res1", "Seq2", "Pos2", "Res2", "Label"])
df.to_csv(output_file, index=False)
print(f"Saved {len(df)} aligned residue pairs to {output_file}")


Saved 16399750 aligned residue pairs to clean_ground_truth.csv


In [4]:
#INPUT SEQUENCES FORMATTER
%pip install biopython
from Bio import SeqIO
import json
from Bio import SeqIO
sequences = {
    record.id.split('|')[2]: str(record.seq) 
    for record in SeqIO.parse("all_sequences.fasta", "fasta")
}
with open("sequences.json", "w") as f:
    json.dump(sequences, f)

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pickle

def split_ground_truth_by_pair(csv_path):
    """
    Splits the ground truth alignment CSV into a dictionary 
    where each key is a (Seq1, Seq2) tuple and each value is a DataFrame 
    containing the corresponding aligned residue pairs.

    Args:
        csv_path (str): Path to the clean_ground_truth.csv file.
        
    Returns:
        dict: { (Seq1, Seq2): DataFrame of corresponding alignments }
    """
    df = pd.read_csv(csv_path)

    df['pair'] = list(zip(df['Seq1'], df['Seq2']))

    pair_to_df = {pair: group.drop(columns=['pair']).reset_index(drop=True) 
                  for pair, group in df.groupby('pair')}

    return pair_to_df

pair_to_ground_truth = split_ground_truth_by_pair('clean_ground_truth.csv')

print(f"Total number of protein pairs: {len(pair_to_ground_truth)}")
with open('pair_to_ground_truth.pkl', 'wb') as f:
    pickle.dump(pair_to_ground_truth, f)

print("Ground truth pair mapping saved to pair_to_ground_truth.pkl!")

Total number of protein pairs: 73712
Ground truth pair mapping saved to pair_to_ground_truth.pkl!


In [3]:
import json

with open('sequences.json', 'r') as f:
    sequences = json.load(f)

name_map = {}
with open('name_map.txt', 'r') as f:
    for line in f:
        if '->' in line:
            original, mapped = line.strip().split('->')
            original = original.strip()
            mapped = mapped.strip()
            name_map[original] = mapped

missing_names = []

for original_name in name_map.keys():
    if original_name not in sequences:
        missing_names.append(original_name)

print(f"\nTotal missing: {len(missing_names)}")
print("Missing sequence entries:")
for name in missing_names:
    print(name)



Total missing: 494
Missing sequence entries:
AGC_AKT1
AGC_AKT2
AGC_AKT3
AGC_CDC42BPA
AGC_CDC42BPB
AGC_CDC42BPG
AGC_CIT
AGC_DMPK
AGC_GRK1
AGC_GRK2
AGC_GRK3
AGC_GRK4
AGC_GRK5
AGC_GRK6
AGC_GRK7
AGC_LATS1
AGC_LATS2
AGC_MAST1
AGC_MAST2
AGC_MAST3
AGC_MAST4
AGC_MASTL
AGC_PDPK1
AGC_PDPK2P
AGC_PKN1
AGC_PKN2
AGC_PKN3
AGC_PRKACA
AGC_PRKACB
AGC_PRKACG
AGC_PRKCA
AGC_PRKCB
AGC_PRKCD
AGC_PRKCE
AGC_PRKCG
AGC_PRKCH
AGC_PRKCI
AGC_PRKCQ
AGC_PRKCZ
AGC_PRKG1
AGC_PRKG2
AGC_PRKX
AGC_PRKY
AGC_ROCK1
AGC_ROCK2
AGC_RPS6KA1_1
AGC_RPS6KA2_1
AGC_RPS6KA3_1
AGC_RPS6KA4_1
AGC_RPS6KA5_1
AGC_RPS6KA6_1
AGC_RPS6KB1
AGC_RPS6KB2
AGC_RSKR
AGC_SGK1
AGC_SGK2
AGC_SGK3
AGC_STK32A
AGC_STK32B
AGC_STK32C
AGC_STK38
AGC_STK38L
OTHER_AURKA
OTHER_AURKB
OTHER_AURKC
CAMK_BRSK1
CAMK_BRSK2
CAMK_CAMK1
CAMK_CAMK1D
CAMK_CAMK1G
CAMK_CAMK2A
CAMK_CAMK2B
CAMK_CAMK2D
CAMK_CAMK2G
CAMK_CAMK4
OTHER_CAMKK1
OTHER_CAMKK2
CAMK_CAMKV
CAMK_CASK
CAMK_CHEK1
CAMK_CHEK2
CAMK_DAPK1
CAMK_DAPK2
CAMK_DAPK3
CAMK_DCLK1
CAMK_DCLK2
CAMK_DCLK3
CAMK_HUNK
CAMK_KALRN
CAM